In [ ]:
import os
import json
from tqdm import tqdm
from collections import Counter
import numpy as np

import torch
from torch.utils.data import DataLoader
import torch.utils.tensorboard as tensorboard
from seqeval.metrics import classification_report

from utils import set_random_seed, Config, load_ner_config
from dataset import io2df, io2bio, padding, NERDataset
from model import BiLSTM_CRF

%load_ext autoreload
%autoreload 2

set_random_seed(seed=0)

In [ ]:
config = Config('config.yaml')
config.__dict__

In [ ]:
tr_titles = io2df(config.TR_PATH)
va_titles = io2df(config.VA_PATH)

In [ ]:
tr_titles

# Tag -> tagID

In [ ]:
TAG2IDX = load_ner_config('ner_tags/ner_coarse_grained.json')
IDX2TAG = {i: t for t, i in TAG2IDX.items()}

In [ ]:
tr_titles['tags_ids'] = tr_titles['tags'].transform(lambda x: [TAG2IDX[tag] for tag in x])
va_titles['tags_ids'] = va_titles['tags'].transform(lambda x: [TAG2IDX[tag] for tag in x])

# Token -> tokenID

In [ ]:
def calc_token_cntr(filepath):

    token_cntr = Counter()
    num_lines = sum(1 for _ in open(filepath, encoding="utf-8"))

    with open(filepath, "r", encoding="utf-8") as f:
        for line in tqdm(f, total=num_lines):
            line = line.strip().split()
            if line:
                token, fine_tag = line
                token_cntr[token] += 1
    
    return token_cntr

token_cntr = calc_token_cntr(filepath=config.TR_PATH)


MC = 50_000
top_tokens = [token for token, _ in token_cntr.most_common(MC)]
TOKEN2IDX = {token: i + 2 for i, token in enumerate(top_tokens)}
TOKEN2IDX['PAD'] = 0
TOKEN2IDX['UKN'] = 1

with open('tokenizers/token2idx.json', 'w') as f:
    json.dump(TOKEN2IDX, f, indent=4)

In [ ]:
tr_titles['tokens_ids'] = tr_titles['tokens'].transform(lambda x: [TOKEN2IDX[token] if token in TOKEN2IDX else TOKEN2IDX['UKN'] for token in x])
va_titles['tokens_ids'] = va_titles['tokens'].transform(lambda x: [TOKEN2IDX[token] if token in TOKEN2IDX else TOKEN2IDX['UKN'] for token in x])

# Padding

In [ ]:
tr_titles['tokens_ids'] = tr_titles['tokens_ids'].transform(padding, max_len=config.SEQ_LEN)
tr_titles['tags_ids'] = tr_titles['tags_ids'].transform(padding, max_len=config.SEQ_LEN)

va_titles['tokens_ids'] = va_titles['tokens_ids'].transform(padding, max_len=config.SEQ_LEN)
va_titles['tags_ids'] = va_titles['tags_ids'].transform(padding, max_len=config.SEQ_LEN)

In [ ]:
tr_titles

# Data loader

In [ ]:
tr_dataset = NERDataset(tr_titles)
va_dataset = NERDataset(va_titles)

tr_dataloader = DataLoader(dataset=tr_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=4)
va_dataloader = DataLoader(dataset=va_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=4)

tr_dataset.__getitem__(123)

# Model

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = BiLSTM_CRF(
    embed_size     = config.EMBED_SIZE,
    hidden_size    = config.HIDDEN_SIZE, 
    dropout        = config.DROPOUT,
    token_voc_size = len(TOKEN2IDX) + 1, 
    tag_voc_size   = len(TAG2IDX) + 1
).to(device)

In [ ]:
print(model)

# Train

In [ ]:
weights_folder = 'weights'
if not os.path.exists(weights_folder):
    os.makedirs(weights_folder)
    
runs_folder = '.runs'
if not os.path.exists(runs_folder):
    os.makedirs(runs_folder)

In [ ]:
# Make optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)

# Make tensorboard writer
writer = tensorboard.SummaryWriter(log_dir='./runs')

for epoch in range(10):

    # TRAINING PHASE
    
    tr_losses = []
    
    model.train()
    for tr_batch in tqdm(tr_dataloader, total=tr_dataloader.__len__()):
        optimizer.zero_grad()
        
        tr_xs = tr_batch['tokens_ids'].to(device)
        tr_ys = tr_batch['tags_ids'].to(device)
        
        # Calculate loss
        tr_emission_scores = model(tr_xs).to(device) # size: [batch=128, seq_len=100, 10]
        tr_loss = model.loss_fn(emission_scores=tr_emission_scores, tags=tr_ys, mask=(tr_ys > 0).bool())
        tr_losses.append(tr_loss.item())
        
        # Calculate total loss
        total_loss = tr_loss + model.regularization_loss_fn(lam=1e-3, alpha=0.5)
        
        # Backward pass: compute gradient of the loss w.r.t. all learnable parameters
        total_loss.backward()
        
        # Clip computed gradients
        grad_norm = torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1e2)
        
        # Optimize: update the weights using Adam optimizer
        optimizer.step()
        
    # END TRAINING PHASE AND UPDATE LOG

    with torch.no_grad():
        print(f"Epoch: {epoch:02d} NLL:{tr_loss.item()}")
        writer.add_scalar('tr/'+'loss', np.mean(tr_losses), global_step=epoch)
        writer.add_scalar('tr/'+'total_grad_norm', grad_norm, global_step=epoch)
        for name, param in model.named_parameters():
            writer.add_histogram('tr/'+name, param.data, global_step=epoch)
        print("tr loss", np.mean(tr_losses))
        
    # VALIDATION PHASE
    
    va_losses = []
    
    batch_preds = []
    batch_trues = []

    model.eval()
    with torch.no_grad():
        for va_batch in tqdm(va_dataloader, total=va_dataloader.__len__()):
            va_xs = va_batch['tokens_ids'].to(device) # size: [batch=128, seq_len=100]
            va_ys = va_batch['tags_ids'].to(device) # size: [batch=128, seq_len=100]

            # Forward pass: compute predicted output by passing input to the model
            va_emission_scores = model(va_xs).to(device) # size: [batch=128, seq_len=100]
            va_preds = torch.tensor(model.decode(va_emission_scores))
            va_loss = model.loss_fn(emission_scores=va_emission_scores, tags=va_ys, mask=(va_ys > 0).bool())
            va_losses.append(va_loss.item())
            

            mask = (va_ys > 0).bool()

            for row_id, true in enumerate(va_ys):
                # do not count padding
                true_tags = true[mask[row_id]]
                # idx2tag
                true_tags = [IDX2TAG[idx] for idx in true_tags.tolist()]
                # convert to the format expected by seqeval
                true_tags = io2bio(true_tags)
                batch_trues.append(true_tags)

            for row_id, pred in enumerate(va_preds):
                # do not count padding
                pred_tags = pred[mask[row_id]]
                # idx2tag
                pred_tags = [IDX2TAG[idx] for idx in pred_tags.tolist()]
                # convert to the format expected by seqeval
                pred_tags = io2bio(pred_tags)
                batch_preds.append(pred_tags)
            

        for i in range(5):
            print('pred:', batch_preds[i])
            print('true:', batch_trues[i])
            print()

        print("va loss", np.mean(va_losses))
        writer.add_scalar('va/'+'loss', np.mean(va_losses), global_step=epoch)

        report = classification_report(y_true=batch_trues, y_pred=batch_preds, zero_division=0)
        print(report)

    torch.save(model.state_dict(), f"weights/model_epoch_{epoch:02d}.pt")

writer.close()

In [ ]:
# 100%|██████████| 1030/1030 [01:50<00:00,  9.36it/s]
# Epoch: 00 NLL:445.9901428222656
# tr loss 1491.743241008277
#   0%|          | 0/148 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorCompare.cpp:493.)
#   score = torch.where(mask[i].unsqueeze(1), next_score, score)
# 100%|██████████| 148/148 [00:44<00:00,  3.31it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'B-location', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'B-location', 'O']

# pred: ['O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O']
# true: ['O', 'B-person', 'I-person', 'O', 'B-organization', 'I-organization', 'I-organization', 'O']

# va loss 838.3352361112028
#               precision    recall  f1-score   support

#          art       0.49      0.48      0.49      2063
#     building       0.58      0.42      0.49      2484
#        event       0.46      0.41      0.43      2034
#     location       0.66      0.75      0.71     13649
# organization       0.54      0.47      0.50      9585
#        other       0.33      0.39      0.36      4958
#       person       0.76      0.75      0.76     10954
#      product       0.37      0.06      0.11      2955

#    micro avg       0.60      0.57      0.59     48682
#    macro avg       0.52      0.47      0.48     48682
# weighted avg       0.59      0.57      0.57     48682

# 100%|██████████| 1030/1030 [01:49<00:00,  9.38it/s]
# Epoch: 01 NLL:320.928955078125
# tr loss 709.1378007611024
# 100%|██████████| 148/148 [00:44<00:00,  3.35it/s]
# pred: ['B-person', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['B-person', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'O', 'B-building', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O']

# va loss 657.1076351371971
#               precision    recall  f1-score   support

#          art       0.60      0.59      0.60      2063
#     building       0.61      0.52      0.56      2484
#        event       0.56      0.50      0.53      2034
#     location       0.71      0.77      0.74     13649
# organization       0.61      0.52      0.56      9585
#        other       0.55      0.37      0.44      4958
#       person       0.76      0.82      0.79     10954
#      product       0.54      0.32      0.40      2955

#    micro avg       0.67      0.63      0.65     48682
#    macro avg       0.62      0.55      0.58     48682
# weighted avg       0.66      0.63      0.64     48682

# 100%|██████████| 1030/1030 [01:47<00:00,  9.55it/s]
# Epoch: 02 NLL:192.8609619140625
# tr loss 539.4788638513065
# 100%|██████████| 148/148 [00:44<00:00,  3.36it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'O']

# pred: ['B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'B-other', 'I-other', 'O']
# true: ['B-other', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O']

# va loss 596.615659765295
#               precision    recall  f1-score   support

#          art       0.69      0.59      0.64      2063
#     building       0.65      0.54      0.59      2484
#        event       0.64      0.49      0.56      2034
#     location       0.75      0.77      0.76     13649
# organization       0.62      0.56      0.59      9585
#        other       0.51      0.53      0.52      4958
#       person       0.82      0.78      0.80     10954
#      product       0.52      0.43      0.47      2955

#    micro avg       0.69      0.66      0.67     48682
#    macro avg       0.65      0.59      0.62     48682
# weighted avg       0.69      0.66      0.67     48682

# 100%|██████████| 1030/1030 [01:48<00:00,  9.49it/s]
# Epoch: 03 NLL:204.86279296875
# tr loss 444.1378214715754
# 100%|██████████| 148/148 [00:45<00:00,  3.24it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'B-art', 'I-art', 'O', 'O', 'O', 'B-organization', 'O', 'B-organization', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'B-organization', 'O', 'B-organization', 'O', 'O']

# pred: ['O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'B-event', 'I-event', 'O', 'B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-art', 'I-art', 'I-art', 'I-art', 'O', 'O', 'B-location', 'O', 'O', 'B-person', 'I-person', 'O']
# true: ['O', 'B-art', 'I-art', 'I-art', 'O', 'O', 'B-location', 'O', 'O', 'B-person', 'I-person', 'O']

# pred: ['B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O']
# true: ['B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'I-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'I-event', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O', 'O', 'B-event', 'O', 'O', 'O', 'O']

# pred: ['O', 'B-product', 'I-product', 'O', 'O', 'O', 'O', 'B-product', 'I-product', 'I-product', 'I-product', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'I-product', 'I-product', 'O', 'O']
# true: ['O', 'B-product', 'I-product', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'I-product', 'I-product', 'O', 'O']

# va loss 556.4031047821045
#               precision    recall  f1-score   support

#          art       0.69      0.63      0.66      2063
#     building       0.65      0.53      0.59      2484
#        event       0.65      0.51      0.57      2034
#     location       0.76      0.76      0.76     13649
# organization       0.61      0.60      0.60      9585
#        other       0.56      0.53      0.54      4958
#       person       0.82      0.80      0.81     10954
#      product       0.54      0.46      0.50      2955

#    micro avg       0.70      0.67      0.68     48682
#    macro avg       0.66      0.60      0.63     48682
# weighted avg       0.70      0.67      0.68     48682

# 100%|██████████| 1030/1030 [01:48<00:00,  9.47it/s]
# Epoch: 04 NLL:226.917236328125
# tr loss 379.3760852702613
# 100%|██████████| 148/148 [00:44<00:00,  3.35it/s]
# pred: ['O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'I-product', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'B-product', 'I-product', 'O', 'B-location', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'B-building', 'I-building', 'I-building', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O']
# true: ['O', 'O', 'O', 'B-building', 'I-building', 'I-building', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O']

# pred: ['O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'I-other', 'I-other', 'O']
# true: ['O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# va loss 548.5204033722748
#               precision    recall  f1-score   support

#          art       0.71      0.61      0.66      2063
#     building       0.64      0.55      0.59      2484
#        event       0.61      0.52      0.56      2034
#     location       0.74      0.79      0.76     13649
# organization       0.68      0.54      0.60      9585
#        other       0.58      0.52      0.55      4958
#       person       0.81      0.81      0.81     10954
#      product       0.55      0.46      0.50      2955

#    micro avg       0.71      0.67      0.69     48682
#    macro avg       0.67      0.60      0.63     48682
# weighted avg       0.70      0.67      0.68     48682

# 100%|██████████| 1030/1030 [01:47<00:00,  9.56it/s]
# Epoch: 05 NLL:213.91635131835938
# tr loss 332.2599131315657
# 100%|██████████| 148/148 [00:44<00:00,  3.34it/s]
# pred: ['O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'O']
# true: ['O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O']

# pred: ['B-person', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O']
# true: ['B-person', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'B-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'I-person', 'O']

# pred: ['O', 'O', 'O', 'O', 'B-product', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'B-location', 'I-location', 'O']
# true: ['O', 'O', 'O', 'O', 'B-product', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'B-location', 'O']

# va loss 535.9771515098778
#               precision    recall  f1-score   support

#          art       0.68      0.66      0.67      2063
#     building       0.64      0.55      0.59      2484
#        event       0.58      0.55      0.57      2034
#     location       0.77      0.77      0.77     13649
# organization       0.64      0.59      0.61      9585
#        other       0.61      0.48      0.54      4958
#       person       0.81      0.82      0.82     10954
#      product       0.57      0.46      0.51      2955

#    micro avg       0.71      0.67      0.69     48682
#    macro avg       0.66      0.61      0.63     48682
# weighted avg       0.71      0.67      0.69     48682

# 100%|██████████| 1030/1030 [01:47<00:00,  9.56it/s]
# Epoch: 06 NLL:128.99571228027344
# tr loss 297.88434406761985
# 100%|██████████| 148/148 [00:45<00:00,  3.27it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'B-organization', 'I-organization', 'O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'B-person', 'O', 'O', 'B-organization', 'O', 'B-person', 'I-person', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'B-organization', 'I-organization', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'B-organization', 'O', 'B-organization', 'I-organization', 'O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'B-person', 'O', 'O', 'B-organization', 'O', 'B-person', 'I-person', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O', 'O', 'B-person', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'O']

# va loss 551.6463790068756
#               precision    recall  f1-score   support

#          art       0.76      0.57      0.66      2063
#     building       0.61      0.56      0.59      2484
#        event       0.64      0.53      0.58      2034
#     location       0.73      0.79      0.76     13649
# organization       0.62      0.60      0.61      9585
#        other       0.62      0.49      0.55      4958
#       person       0.83      0.79      0.81     10954
#      product       0.58      0.45      0.51      2955

#    micro avg       0.71      0.67      0.69     48682
#    macro avg       0.68      0.60      0.63     48682
# weighted avg       0.70      0.67      0.68     48682

# 100%|██████████| 1030/1030 [01:47<00:00,  9.55it/s]
# Epoch: 07 NLL:105.38456726074219
# tr loss 268.6778281498881
# 100%|██████████| 148/148 [00:44<00:00,  3.35it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-person', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['B-organization', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'O']

# pred: ['O', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-art', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'B-art', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-art', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# va loss 573.5076387766245
#               precision    recall  f1-score   support

#          art       0.67      0.66      0.66      2063
#     building       0.65      0.52      0.58      2484
#        event       0.65      0.52      0.58      2034
#     location       0.77      0.76      0.77     13649
# organization       0.63      0.59      0.61      9585
#        other       0.60      0.51      0.55      4958
#       person       0.75      0.84      0.79     10954
#      product       0.57      0.47      0.52      2955

#    micro avg       0.70      0.68      0.69     48682
#    macro avg       0.66      0.61      0.63     48682
# weighted avg       0.69      0.68      0.68     48682

# 100%|██████████| 1030/1030 [01:47<00:00,  9.55it/s]
# Epoch: 08 NLL:117.54964447021484
# tr loss 244.04071933042655
# 100%|██████████| 148/148 [00:44<00:00,  3.34it/s]
# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'O']
# true: ['B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'B-organization', 'I-organization', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'I-building', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-building', 'I-building', 'I-building', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O']
# true: ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-event', 'I-event', 'I-event', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O']

# va loss 583.5407335436022
#               precision    recall  f1-score   support

#          art       0.67      0.66      0.67      2063
#     building       0.60      0.56      0.58      2484
#        event       0.57      0.53      0.55      2034
#     location       0.76      0.77      0.76     13649
# organization       0.64      0.58      0.61      9585
#        other       0.59      0.51      0.55      4958
#       person       0.79      0.82      0.81     10954
#      product       0.58      0.43      0.50      2955

#    micro avg       0.70      0.67      0.69     48682
#    macro avg       0.65      0.61      0.63     48682
# weighted avg       0.70      0.67      0.68     48682

# 100%|██████████| 1030/1030 [01:49<00:00,  9.44it/s]
# Epoch: 09 NLL:114.46092224121094
# tr loss 222.36769782501517
# 100%|██████████| 148/148 [00:44<00:00,  3.35it/s]
# pred: ['O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# pred: ['B-person', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']
# true: ['B-person', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'I-organization', 'O', 'B-organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-organization', 'I-organization', 'O']

# pred: ['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O']

# pred: ['O', 'O', 'O', 'O', 'O', 'O']
# true: ['O', 'O', 'O', 'O', 'O', 'O']

# va loss 610.0239699595684
#               precision    recall  f1-score   support

#          art       0.68      0.65      0.66      2063
#     building       0.61      0.56      0.58      2484
#        event       0.61      0.53      0.57      2034
#     location       0.75      0.77      0.76     13649
# organization       0.60      0.61      0.60      9585
#        other       0.57      0.53      0.55      4958
#       person       0.81      0.80      0.80     10954
#      product       0.55      0.45      0.50      2955

#    micro avg       0.69      0.68      0.68     48682
#    macro avg       0.65      0.61      0.63     48682
# weighted avg       0.69      0.68      0.68     48682

In [19]:
# !tensorboard --logdir=runs